In [1]:
import pandas as pd
import pymongo
import numpy as np

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [6]:
reactome = pd.read_table("../data/Reactome/NCBI2Reactome_All_Levels.txt",header=None,low_memory=False)

In [54]:
data = {"gene_id":[],"acc":[],"pmid":[]}

In [55]:
for tup in reactome[[0,1,4]].itertuples():
    pmids = eval(tup._3)
    data["acc"]+=[tup._2]*len(pmids)
    data["gene_id"]+=[tup._1]*len(pmids)
    data["pmid"]+=pmids

In [56]:
data = pd.DataFrame(data)

In [57]:
data = data[data.gene_id.str.isnumeric()].copy()

In [58]:
data.gene_id = data.gene_id.astype(int)

In [60]:
to_insert = []
for i,row in data.iterrows():
    to_insert.append(dict(row))
    if i%25000==0:
        db.reactome.insert_many(to_insert)
        to_insert = []
db.reactome.insert_many(to_insert)